# Project 1

Due 9/16

## Part 1: word counting and bag of words

In [ ]:
!pip install spacy


In [ ]:
!python -m spacy download en_core_web_sm



In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import spacy
language_model = spacy.load("en_core_web_sm")